# Fine-Tuning du Modèle LLAMA 3

Ce notebook a pour objectif d'effectuer un fine-tuning du modèle **LLAMA 3** en utilisant un dataset personnalisé.  
Nous allons suivre les étapes suivantes :

1. **Installation des Dépendances** : Installation des bibliothèques nécessaires, notamment `transformers`, `bitsandbytes`, `datasets` et `peft`.  
2. **Connexion à Hugging Face et Téléchargement du Modèle** : Authentification à Hugging Face et chargement du modèle [`Llama-3.2-1B-Instruct`](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct).  
3. **Traitement du Jeu de Données** : Prétraitement des données, tokenization et mise en forme pour le fine-tuning.  
4. **Entraînement du Modèle** : Fine-tuning en utilisant **LoRA** et **QLoRA** pour optimiser la consommation mémoire.  
5. **Sauvegarde du Modèle et Conversion vers Ollama** : Exportation du modèle entraîné au format compatible **Ollama** pour une utilisation simplifiée en local.  

Nous utiliserons **`transformers`** de Hugging Face pour le modèle, **`datasets`** pour la gestion des données, et **`peft`** pour des optimisations d'entraînement sur GPU.

💡 **Objectif** : Adapter LLAMA 3 à une tâche spécifique tout en optimisant les performances et la consommation mémoire.

🚀 **Commençons !**

# 1. Installation des Dépendances

Avant de commencer le fine-tuning de LLAMA 3, assurez-vous d’installer les logiciels recommandés :

✅ **Git Bash** → [Télécharger ici](https://git-scm.com/downloads)  
✅ **Python 3.12** → [Télécharger ici](https://www.python.org/downloads/release/python-3128/)  
📌 *Pensez à cocher l’option "Ajouter Python aux variables d’environnement" lors de l’installation*  
✅ **VS Code** → [Télécharger ici](https://code.visualstudio.com/Download)  
✅ **Ollama** → [Télécharger ici](https://ollama.com/download)  

---

### 📌 Exécution du script d’installation

Une fois les logiciels installés, ouvrez un terminal **Git Bash** et exécutez la commande suivante pour lancer l’installation des dépendances et du projet :

   ```bash
   bash installation.sh





# 2. Connexion à Hugging Face

Pour télécharger et utiliser le modèle **Llama-3.2-1B-Instruct**, vous devez d’abord configurer votre accès à Hugging Face.

### 🔹 Étape 1 : Créer un compte Hugging Face
Si vous n’avez pas encore de compte, créez-en un ici :  
👉 [Créer un compte Hugging Face](https://huggingface.co/join)

---

### 🔹 Étape 2 : Générer un Token d’authentification
1. Rendez-vous dans les paramètres des tokens :  
   👉 [Créer un token ici](https://huggingface.co/settings/tokens)
2. Cliquez sur **New token**.
3. Donnez-lui un nom (ex. : `llama3-token`) et attribuez-lui les droits **read**.
4. Copiez le token généré.

---

### 🔹 Étape 3 : Demander l'accès au modèle LLAMA 3  
Le modèle **Llama-3.2-1B-Instruct** nécessite une demande d'accès.  
Rendez-vous sur cette page et cliquez sur **Request access** :  
👉 [Demander l’accès au modèle](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct)  

---

### 🔹 Étape 4 : Connexion à Hugging Face dans le Notebook  

Exécutez le code suivant pour vous connecter à Hugging Face avec votre token dans le terminal :
   ```
   huggingface-cli login
   ```
---

### 🔹 Étape 5 : Téléchargement et Chargement du Modèle LLAMA 3  

Une fois connecté, on peut télécharger et charger le modèle :
      ```

      from transformers import AutoModelForCausalLM, AutoTokenizer

      MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"


      ### Chargement du tokenizer et du modèle
      tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
      model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

      print("✅ Modèle LLAMA 3 chargé avec succès !")

      ```
🚀 **Votre modèle LLAMA 3 est maintenant prêt à être utilisé !**



In [1]:
# Use a pipeline as a high-level helper
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

c:\Users\Quera\Desktop\INRAE-Llama3\.llama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt = "Qui-es-tu ?"

# Tokeniser l'entrée
inputs = tokenizer(prompt, return_tensors="pt")

In [3]:
# Visualisation des tokens d'entrée
outputs =model.generate(**inputs, max_length=40)

outputs

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [4]:
# Visualisation des tokens de sorties
outputs =model.generate(**inputs, max_length=40)

outputs

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [5]:
# Décodage des tokens de sortie
tokenizer.decode(outputs[0], skip_special_tokens=True)



'Qui-es-tu? (I am you)\nI am you\nI am you\n\nUn peu de philosophie\n\nUn être humain est une forme de vie complexe, avec des bes'

In [6]:
# Sauvegarde du modèle 
model.save_pretrained("llama3-1B")
tokenizer.save_pretrained("llama3-1B")

('llama3-1B\\tokenizer_config.json',
 'llama3-1B\\special_tokens_map.json',
 'llama3-1B\\tokenizer.json')

In [8]:
# Conversion en gguf pour utilisation dans Ollama

!python ./llama.cpp/convert_hf_to_gguf.py ./llama3-1B

INFO:hf-to-gguf:Loading model: llama3-1B
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> F16, shape = {2048, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float32 --> F16, shape = {8192, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> F16, shape = {2048, 512}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float32 --> F16, shape = {2048, 2048}
INFO:hf-to-g

# 3. Traitement du Jeu de Données

## 📌 Format des données  

Le modèle **LLAMA 3** attend des données sous un format spécifique.  
Nous devons structurer les conversations en **JSON Lines** (`.jsonl`), où chaque ligne représente un échange entre l’utilisateur et l’assistant.

Exemple de fichier `data.jsonl` :

```json
{"messages": [{"role": "user", "content": "Qui es-tu ?"}, {"role": "assistant", "content": "Je suis l'assistant de l'INRAE<|endoftext|>"}]}
```

### 🔹 Explication :  
- **`role: user`** → Contenu du message de l’utilisateur  
- **`role: assistant`** → Réponse générée par le modèle  
- **`<|endoftext|>`** → **Token de fin de texte** qui marque la fin d’une conversation  

Ce format est essentiel pour que le modèle apprenne à répondre correctement en conservant le contexte.

---

## 🔄 Transformations appliquées  

Le modèle **LLAMA 3** ne peut pas directement utiliser du texte brut. Nous devons **convertir** les messages en une forme que le modèle peut traiter :  
- **`input_ids`** : Représentation des tokens du texte  
- **`attention_mask`** : Masque indiquant quelles parties du texte doivent être prises en compte  
- **`labels`** : Données de sortie attendues pour l’apprentissage  

### 📍 **Avant transformation :**  
Format brut des données :

```python
{
    "messages": [
        {"role": "user", "content": "Qui es-tu ?"},
        {"role": "assistant", "content": "Je suis l'assistant de l'INRAE<|endoftext|>"}
    ]
}
```

---

### 📍 **Après transformation :**  
Format prêt pour le fine-tuning du modèle :

```python
{
    "messages": [
        {"role": "user", "content": "Qui es-tu ?"},
        {"role": "assistant", "content": "Je suis l'assistant de l'INRAE<|endoftext|>"}
    ],
    "input_ids": [128000, 62545, 1560, 2442, 84, 949, 30854, 36731, 326, 6, 78191, 409, 326, 6, 691, 5726, 36, 27, 91, 8862, 728, 428, 91, 29, 128009],
    "attention_mask": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    "labels": [128000, 62545, 1560, 2442, 84, 949, 30854, 36731, 326, 6, 78191, 409, 326, 6, 691, 5726, 36, 27, 91, 8862, 728, 428, 91, 29, -100]
}
```

### 🔹 **Explication des transformations :**  
1. **Tokenization** :  
   - Le texte est converti en **`input_ids`**, une séquence de nombres représentant chaque mot ou caractère.  
   - Exemple : `"Qui es-tu ?"` → `[128000, 62545, 1560, 2442, 84]`  

2. **Masking** (**`attention_mask`**) :  
   - Indique quelles parties du texte doivent être prises en compte.  
   - `1` = Token important, `0` = Token ignoré (padding).  

3. **Labels pour l’entraînement** (**`labels`**) :  
   - Même séquence que **`input_ids`**, sauf que les tokens d’entrée utilisateur sont remplacés par `-100` (pour ignorer leur contribution à la prédiction).  
   - Cela permet au modèle de **prédire uniquement la réponse** et d’ignorer l’entrée utilisateur.  

---

In [7]:
from datasets import load_dataset

# ✅ Charger le dataset JSONL
dataset = load_dataset('json', data_files='./data.jsonl')

dataset

Generating train split: 4 examples [00:00, 137.62 examples/s]


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 4
    })
})

In [ ]:
# ✅ Vérifier les clés du dataset
print("Exemple de données :", dataset['train'][0])  # Affiche un exemple pour vérifier sa structure


Exemple de données : {'messages': [{'role': 'user', 'content': 'Qui es-tu ?'}, {'role': 'assistant', 'content': "Je suis l'assistant de l'INRAE<|endoftext|>"}]}


In [ ]:
# Création du token de padding pour Lama3
tokenizer.pad_token = tokenizer.eos_token  

In [ ]:
# Prétraitement des données d'entrainement
def preprocess_function(examples):
    # 🔹 Construire le texte d'entrée à partir des messages
    text_inputs = []
    for message_set in examples["messages"]:
        text = ""
        for message in message_set:
            text += f"{message['content']}"
        text_inputs.append(text.strip())

    # 🔹 Tokenisation
    inputs = tokenizer(text_inputs, truncation=True, padding="max_length", max_length=25)

    # 🔹 Copier input_ids pour labels
    inputs["labels"] = inputs["input_ids"].copy()

    # 🔹 Remplacer les tokens de padding par -100 pour ignorer la perte
    padding_token_id = tokenizer.pad_token_id
    inputs["labels"] = [
        [(label if label != padding_token_id else -100) for label in labels] for labels in inputs["labels"]
    ]

    return inputs

# ✅ Appliquer la transformation
dataset = dataset.map(preprocess_function, batched=True)
print("Exemple de données :", dataset['train'][0])  # Affiche un exemple pour vérifier sa structure


Map: 100%|██████████| 4/4 [00:00<00:00, 172.96 examples/s]

Exemple de données : {'messages': [{'role': 'user', 'content': 'Qui es-tu ?'}, {'role': 'assistant', 'content': "Je suis l'assistant de l'INRAE<|endoftext|>"}], 'input_ids': [128000, 62545, 1560, 2442, 84, 949, 30854, 36731, 326, 6, 78191, 409, 326, 6, 691, 5726, 36, 27, 91, 8862, 728, 428, 91, 29, 128009], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], 'labels': [128000, 62545, 1560, 2442, 84, 949, 30854, 36731, 326, 6, 78191, 409, 326, 6, 691, 5726, 36, 27, 91, 8862, 728, 428, 91, 29, -100]}


# 4. Entraînement du Modèle

Après avoir préparé nos données, nous pouvons maintenant entraîner **LLAMA 3**.  
Nous utiliserons **LoRA (Low-Rank Adaptation)** pour optimiser l'entraînement tout en **réduisant la charge mémoire**.

---

### 📌 4.1 Configuration du Fine-Tuning avec LoRA

### ✅ **Pourquoi LoRA ?**
Le fine-tuning complet d'un modèle **LLAMA 3** peut être très gourmand en ressources GPU.  
LoRA permet de **réduire la quantité de poids entraînables** en n'entraînant que certaines couches du modèle.

Nous définissons les paramètres de LoRA avec :

```python
from peft import LoraConfig, get_peft_model

# ✅ Config LoRA
lora_config = LoraConfig(
    r=32, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)
```

### 📍 **Explication des paramètres :**
- **`r=32`** → Taille du rang pour la factorisation des matrices. Plus il est élevé, plus l'entraînement est précis, mais coûteux.  
- **`lora_alpha=32`** → Facteur de mise à l'échelle des poids LoRA.  
- **`target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]`** → On applique LoRA uniquement sur certaines couches du modèle.  
- **`lora_dropout=0.05`** → Ajout d'un **dropout** pour éviter l'overfitting.  
- **`task_type="CAUSAL_LM"`** → On entraîne un modèle **causal** (génération de texte).  

Nous appliquons ensuite cette configuration au modèle :

```python
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
```

---

### 📌 4.2 Définition des Paramètres d'Entraînement

Nous utilisons **`TrainingArguments`** pour définir les options d'entraînement :

```python
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama3-finetuned",
    per_device_train_batch_size=5,  # ⚠️ Réduction de la batch size car CPU limité
    num_train_epochs=10,  # 🔹 10 époques pour un bon fine-tuning
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-3,
    gradient_accumulation_steps=90,  # 🔹 Augmenter pour réduire la charge mémoire
    fp16=False,  # 🚫 Désactivé car inutile sur CPU
    bf16=False,
    gradient_checkpointing=False,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    save_total_limit=2,  # 🔹 Garde seulement les 2 derniers checkpoints
    weight_decay=0.01,
    report_to="tensorboard",  # 🔹 Enregistre les logs pour TensorBoard
    torch_compile=False,  # ✅ Optimisation CPU
    no_cuda=True  # 🚫 Désactive l'utilisation du GPU
)
```

### 📍 **Explication des paramètres :**
- **`output_dir="./llama3-finetuned"`** → Sauvegarde du modèle entraîné ici.  
- **`per_device_train_batch_size=5`** → On utilise une petite batch size pour éviter d’épuiser la RAM.  
- **`num_train_epochs=10`** → Nombre total de passes sur les données.  
- **`gradient_accumulation_steps=90`** → Permet d’accumuler les gradients sur plusieurs batchs avant de mettre à jour les poids (réduit la charge mémoire).  
- **`learning_rate=2e-3`** → Taux d’apprentissage initial.  
- **`lr_scheduler_type="cosine"`** → Programmation du taux d’apprentissage en fonction d’une courbe **cosinus**.  
- **`save_total_limit=2`** → Évite d'enregistrer trop de checkpoints pour économiser de l'espace disque.  
- **`torch_compile=False`** → Optimisation de l’entraînement pour CPU uniquement.  
- **`no_cuda=True`** → **Forçage du CPU** (utile si aucun GPU disponible).  

---

### 📌 4.3 Préparation des Données pour l'Entraînement

Nous utilisons **`DataCollatorForSeq2Seq`** pour normaliser les données avant l'entraînement.

```python
from transformers import DataCollatorForSeq2Seq

# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt")
```

📍 **Pourquoi ?**  
- Permet d'ajouter du **padding** pour que toutes les séquences aient la même longueur.  
- Convertit les données en **tensors PyTorch** (`return_tensors="pt"`).  

---

### 📌 4.4 Lancement de l’Entraînement  

Enfin, nous utilisons **`Trainer`** pour entraîner le modèle avec nos données :

```python
from transformers import Trainer

# ✅ Entraînement
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['train'],  # 🔹 On utilise le même dataset pour l’évaluation (peut être remplacé)
    data_collator=data_collator
)

trainer.train()
```

### 📍 **Explication :**
- **`model=model`** → Modèle LoRA prêt à être entraîné.  
- **`args=training_args`** → Paramètres d'entraînement définis plus tôt.  
- **`train_dataset=dataset['train']`** → Données utilisées pour l'entraînement.  
- **`eval_dataset=dataset['train']`** → Ici, on utilise le même dataset pour l’évaluation, mais idéalement il faut un dataset séparé.  
- **`data_collator=data_collator`** → Regroupe les données de manière efficace.  

---

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq


# ✅ Config LoRA
lora_config = LoraConfig(
    r=32, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)


print("---------------------------------------------------------------")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("---------------------------------------------------------------")

training_args = TrainingArguments(
    output_dir="./llama3-finetuned",
    per_device_train_batch_size=5,  # ⚠️ Réduire la batch size car CPU est limité
    num_train_epochs=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-3,
    gradient_accumulation_steps=90,  # 🔹 Augmenter pour réduire la charge mémoire
    fp16=False,  # 🚫 Désactiver fp16 (inutile sur CPU)
    bf16=False,
    gradient_checkpointing=False,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    save_total_limit=2,
    weight_decay=0.01,
    report_to="tensorboard",
    torch_compile=False,  # ✅ Optimisation CPU
    no_cuda=True
)


# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt")


# ✅ Entraînement
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['train'],
    data_collator=data_collator
)

trainer.train()

---------------------------------------------------------------
trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5485
---------------------------------------------------------------


c:\Users\Quera\Desktop\Llama3\.llama\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\Quera\Desktop\Llama3\.llama\Lib\site-packages\transformers\training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,3.148244
2,No log,3.027769
3,No log,2.843416
4,No log,2.643522
5,No log,2.435893
6,No log,2.198648
7,No log,1.951529
8,No log,1.705722
9,No log,1.452125
10,No log,1.106727


TrainOutput(global_step=10, training_loss=2.3531587600708006, metrics={'train_runtime': 60.1425, 'train_samples_per_second': 0.665, 'train_steps_per_second': 0.166, 'total_flos': 7526107054080.0, 'train_loss': 2.3531587600708006, 'epoch': 10.0})

In [ ]:
# Deuxième série d'entrainement
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.106727
2,No log,1.055463
3,No log,0.946546
4,No log,0.868594
5,No log,0.784750
6,No log,0.658026
7,No log,0.600254
8,No log,0.588399
9,No log,0.586216
10,No log,0.585886


TrainOutput(global_step=10, training_loss=0.7959489822387695, metrics={'train_runtime': 51.1493, 'train_samples_per_second': 0.782, 'train_steps_per_second': 0.196, 'total_flos': 7526107054080.0, 'train_loss': 0.7959489822387695, 'epoch': 10.0})

In [ ]:
# Transformation des tokens d'entré
prompt ="Qui es-tu ?"
input_exemple = tokenizer(prompt, return_tensors="pt")
input_exemple

{'input_ids': tensor([[128000,  62545,   1560,   2442,     84,    949]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
# Prédiction du modèle après entrainement
output = model.generate(**input_exemple, max_length=25)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
# Transformation des tokens de sortie
tokenizer.decode(output[0], skip_special_tokens=True)

"Qui es-tu?Je suis l'assistant de l'INRAE<|endoftext|>Je"

# 5. Sauvegarde du Modèle et Conversion vers Ollama

Une fois l’entraînement terminé, nous devons **enregistrer** notre modèle pour pouvoir l’utiliser ultérieurement.  

Deux étapes sont nécessaires :
1. **Sauvegarde du modèle entraîné avec LoRA**
2. **Fusion des poids LoRA avec le modèle de base** pour obtenir un modèle autonome.

---

### 📌 5.1 Sauvegarde du Modèle LoRA

Le modèle fine-tuné avec **LoRA** contient des poids adaptatifs séparés.  
Nous devons sauvegarder :
- **Le modèle LoRA**
- **Le tokenizer** (indispensable pour la génération de texte)

```python
# Sauvegarde du modèle LoRA et du tokenizer
model.save_pretrained("llama3-finetuned")
tokenizer.save_pretrained("llama3-finetuned")
```

📍 **Explication :**  
- `model.save_pretrained("llama3-finetuned")` → Sauvegarde le modèle entraîné avec les poids LoRA.  
- `tokenizer.save_pretrained("llama3-finetuned")` → Sauvegarde le **tokenizer**, essentiel pour la conversion texte → tokens.  

> 📝 **À noter** : À ce stade, le modèle dépend encore des poids **LoRA**.  
> Il faut maintenant **fusionner ces poids** avec le modèle de base.

---

### 📌 5.2 Fusion des Poids LoRA avec le Modèle de Base

LoRA ajoute uniquement des **ajustements** aux poids du modèle de base.  
Si on veut un **modèle autonome**, il faut fusionner ces ajustements avec le modèle d’origine.

```python
from transformers import AutoModelForCausalLM
from peft import PeftModel

# Chargement du modèle de base
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map="cpu")

# Chargement du modèle LoRA et fusion avec le modèle de base
peft_model = PeftModel.from_pretrained(base_model, "llama3-finetuned")
peft_model = peft_model.merge_and_unload()
```

📍 **Explication :**  
1. **On recharge le modèle de base** `meta-llama/Llama-3.2-1B-Instruct`  
2. **On applique les poids LoRA** avec `PeftModel.from_pretrained(base_model, "llama3-finetuned")`  
3. **On fusionne les poids** en appelant `merge_and_unload()`, ce qui intègre définitivement les ajustements LoRA dans le modèle principal.  

Après cette fusion, le modèle devient totalement **indépendant de LoRA** et peut être utilisé comme un modèle standard.

---

## 📌 3. Sauvegarde du Modèle Fusionné

Une fois la fusion terminée, nous enregistrons **le modèle final** :

```python
# Sauvegarde du modèle fusionné (sans LoRA)
peft_model.save_pretrained("llama3-finetuned-merged")
tokenizer.save_pretrained("llama3-finetuned-merged")

print("✅ Fusion et sauvegarde du modèle terminé !")
```

📍 **Explication :**  
- **`peft_model.save_pretrained("llama3-finetuned-merged")`** → Sauvegarde le modèle final, prêt à être utilisé **sans dépendance LoRA**.  
- **`tokenizer.save_pretrained("llama3-finetuned-merged")`** → Sauvegarde le **tokenizer** avec le modèle fusionné.  

---

In [ ]:
# Sauvegarde du modèle LoRA et fusion des poids du modèle de base
from transformers import AutoModelForCausalLM
from peft import PeftModel

model.save_pretrained("llama3-finetuned")
tokenizer.save_pretrained("llama3-finetuned")

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map="cpu")
peft_model = PeftModel.from_pretrained(base_model, "llama3-finetuned")


# Fusionner les poids LoRA avec le modèle principal
peft_model = peft_model.merge_and_unload()

# Sauvegarde du modèle fusionné (sans LoRA)
peft_model.save_pretrained("llama3-finetuned-merged")
tokenizer.save_pretrained("llama3-finetuned-merged")

print("✅ Fusion et sauvegarde du modèle terminé !")

✅ Fusion et sauvegarde du modèle terminé !


In [ ]:
# Conversion en gguf
!python ./llama.cpp/convert_hf_to_gguf.py ./llama3-finetuned-merged

INFO:hf-to-gguf:Loading model: llama3-finetuned-merged
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> F16, shape = {2048, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float32 --> F16, shape = {8192, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> F16, shape = {2048, 512}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float32 --> F16, shape = {2048, 2048

## **6. Conversion du Modèle vers Ollama**

Après avoir fine-tuné et fusionné notre modèle **LLAMA 3**, nous pouvons maintenant le convertir au format **Ollama** pour une exécution optimisée sur **CPU** et **GPU**.

---

### 📌 **Pourquoi utiliser Ollama ?**
**Ollama** est une plateforme permettant d’exécuter des modèles **LLM optimisés** sur des **machines locales**, avec une gestion efficace de la mémoire et du temps d’inférence.

Avantages :
- Exécution **rapide et optimisée** sur CPU/GPU  
- Compatible avec les fichiers **GGUF** (format performant pour LLMs)  
- Facilité d'utilisation avec des **commandes simples**  

---

## **1. Création du Modèle avec Ollama**
Une fois que notre modèle fusionné (`llama3-finetuned-merged`) est prêt, nous devons **créer un modèle Ollama** en utilisant un fichier **Modelfile**.

### 🔹 **Commande pour créer le modèle :**
```bash
ollama create llama-test -f Modelfile
```

📍 **Explication :**
- `ollama create` → Commande pour créer un modèle Ollama  
- `llama-test` → Nom du modèle personnalisé  
- `-f Modelfile` → Spécifie le fichier `Modelfile` qui contient la configuration du modèle  

---

## **2. Structure du Fichier `Modelfile`**
Le fichier `Modelfile` contient les **instructions** nécessaires pour charger et configurer le modèle dans **Ollama**.

```bash
FROM ./llama3-finetuned-merged/Llama-3.2-1B-Instruct-F16.gguf

PARAMETER stop "<|endoftext|>"
```

📍 **Explication des lignes :**
- **`FROM ./llama3-finetuned-merged/Llama-3.2-1B-Instruct-F16.gguf`**  
  - Indique que nous utilisons le modèle fine-tuné, converti au format **GGUF**  
  - Le format **GGUF** est optimisé pour l'exécution rapide sur CPU/GPU  
- **`PARAMETER stop "<|endoftext|>"`**  
  - Définit le **token de fin de texte** `<|endoftext|>`  
  - Permet d'arrêter la génération lorsqu'on atteint ce token  

📌 **Pourquoi GGUF ?**  
Le format **GGUF** (GPT-Generated Unified Format) est une version optimisée des modèles **GGML**, offrant :
- **Meilleure gestion mémoire**
- **Performances accrues sur CPU/GPU**
- **Compatibilité avec Ollama et llama.cpp**

---

## **3. Documentation de `Modelfile`**
Pour plus de détails sur la configuration de `Modelfile`, consultez la documentation officielle d’Ollama :  
🔗 **[Ollama Modelfile Docs](https://github.com/ollama/ollama/blob/main/docs/modelfile.md)**  

In [ ]:
# Création du modèle Ollama
!ollama create llama-test -f Modelfile

gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering mode